In [42]:
import pandas as pd
import os
import numpy as np

In [43]:
import sys
import os

# Get the current folder path
current_folder = os.getcwd()
# Add the parent directory to sys.path
parent_folder = os.path.abspath(os.path.join(current_folder, ".."))
sys.path.append(parent_folder)

# Import the utility module
import utility  # Now you can use utility.py as a module import utility

In [44]:
#import data: 
datapath = parent_folder  + '/Data/'
holidays = pd.read_csv(datapath + 'holidays_events.csv')

In [45]:
# Check how holidays are distributed every year. 
holidays['date'] = pd.to_datetime(holidays['date'])
holidays['year'] = holidays.date.dt.year
holidays['month-day'] = holidays['date'].dt.strftime('%m-%d')

# Show all rows
pd.set_option('display.max_rows', None)
# First: National
national = holidays[(holidays.locale == 'National') & (holidays.type == 'Holiday')]

## Type of holidays 
The different types of holiday indicates for a particular store in a particlular day, it can be:

- holiday, celebrated
- holiday, not celebrated
- not holiday, celebrated 
- not holiday, not celebrated
- event


We don't know if for a particular holiday, whether it's a day off or not is going to affect the sale. So I'll also add "whether it's celebrated (day off)" as a feature. 

## location and description 
There're three locations. Based on the location, we can determine wheter a particular store celebrate the holiday. 

Different holidays will likely have different shopping culture. Holidays should be considered separately. 

So at first glance, I will add three columns:
- Local holiday (0 is none, 1 - 27 is the different discription of the holidays.)
- Regional holiday (0 is None, 1 - 4 is four different holidays.)
- National holiday (0 is None, 1 - 29 are different holidays.)
- National Events (0 is None, 1 - 43 are different events.) 
  - Event is not recruiting. Hence it should be labeled separately. 

One problem, 2012-12-31	is both Puente Primer dia del ano	(Bridge) and Primer dia del ano-1	(Additional)
It termed out Puente Primer dia del ano	means "The bridge for new year" , Primer dia del ano means "new year eve".

Other than that, there's no overlapping holidays. 

Notice that "Bridge", "Work Day" and "Transfer" Changes every year based on the day of the week. 
While "Additional" only goes with the holiday it self. 

# Different holidays 

Not all holiday is repeated in the same day every year. 

In [46]:

table_national = national.pivot_table(index='month-day', columns='year', values='description', aggfunc=lambda x: ', '.join(x)).fillna(0)
# print(table_national)

In [47]:
# Then Regional 
Regional = holidays[(holidays.locale == 'Regional') & (holidays.type == 'Holiday')]
table_regional = Regional.pivot_table(index='month-day', columns='year', values='description', aggfunc=lambda x: ', '.join(x)).fillna(0)
# print(table_regional)

In [48]:
# Finally, local: 

Local = holidays[(holidays.locale == 'Local') & (holidays.type == 'Holiday')]
table_local = Local.pivot_table(index='month-day', columns='year', values='description', aggfunc=lambda x: ', '.join(x)).fillna(0)
# print(table_local)

Not All national holidays are of the same date. \
Regional holidays are of the day date. 

In [49]:
len(national.description.unique())

10

Local holiday is very wild. 
The  Fundacion de Guayaquil is celebrated in different ways every time. 

# Summary on holiday types 

Because how complicated the different holidays are, it's easier to use the holiday.csv as a table and not try to predict the holiday dates. 

Treat every discription separetly. 
To deal with 2012-12-31, label it as "Primer dia del ano" for now. Since new year eve affects more than just a "bridge day". (If any thing goes wrong, this is the "error" we introduce. )



In [50]:


# Create a new dataset: 
Local_holidays = holidays[holidays.locale == 'Local']
Local_holidays['local_holiday'] = Local_holidays['description']
# Set 'Local_celebrated' to True where 'type' is 'Holiday'
Local_holidays['Local_celebrated'] = Local_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
# Update 'Local_celebrated' to False where 'transferred' is True
Local_holidays.loc[Local_holidays['transferred'] == True, 'Local_celebrated'] = False
# Take only the needed 
Local_holidays = Local_holidays[['date', 'local_holiday', 'locale_name','Local_celebrated']]

# For regional too 
Regional_holidays = holidays[holidays.locale == 'Regional']
Regional_holidays['Regional_holiday'] = Regional_holidays['description']
# Set 'Local_celebrated' to True where 'type' is 'Holiday'
Regional_holidays['Regional_celebrated'] = Regional_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
# Update 'Local_celebrated' to False where 'transferred' is True
Regional_holidays.loc[Regional_holidays['transferred'] == True, 'Regional_celebrated'] = False
# Take only the needed 
Regional_holidays = Regional_holidays[['date', 'Regional_holiday', 'locale_name','Regional_celebrated']]

# Same with National 
National_holidays = holidays[holidays.locale == 'National']
National_holidays['National_holiday'] = National_holidays['description']
# Set 'Local_celebrated' to True where 'type' is 'Holiday'
National_holidays['National_celebrated'] = National_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
# Update 'Local_celebrated' to False where 'transferred' is True
National_holidays.loc[National_holidays['transferred'] == True, 'National_celebrated'] = False
# Take only the needed 
National_holidays = National_holidays[['date', 'National_holiday', 'locale_name','National_celebrated']]


/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_99232/1476164347.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Local_holidays['local_holiday'] = Local_holidays['description']
/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_99232/1476164347.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Local_holidays['Local_celebrated'] = Local_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_99232/1476164347.py:1

# Study the effect of holidays 


In [64]:
# Import the combined training file with the correct holiday merge. 
#import data: 
# datapath = os.getcwd() + '/Data/'
#df = pd.read_csv(datapath + 'combined-2.csv')
df = pd.read_csv(datapath + 'merged_train_alt_new.csv')

/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_99232/3265533780.py:5: DtypeWarning: Columns (15,17,19,21,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datapath + 'merged_train_alt_new.csv')


In [65]:
# Uncomment this to see a summary of the df. 
utility.summary(df)
#utility.df_info(df)

data shape: (3000888, 39)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
Unnamed: 0,int64,0,0.000000,3000888,0.0,3000887.0,0,1,2
date,object,0,0.000000,1684,NaN,NaN,2013-01-01,2013-01-01,2013-01-01
year,int64,0,0.000000,5,2013.0,2017.0,2013,2013,2013
month,int64,0,0.000000,12,1.0,12.0,1,1,1
week_number,int64,0,0.000000,53,1.0,53.0,1,1,1
day,int64,0,0.000000,31,1.0,31.0,1,1,1
day_of_week,int64,0,0.000000,7,0.0,6.0,1,1,1
store_nbr,int64,0,0.000000,54,1.0,54.0,1,1,1
type,object,0,0.000000,5,NaN,NaN,D,D,D
cluster,int64,0,0.000000,17,1.0,17.0,13,13,13


In [66]:
df_hol = df[(df.hol_Nat == 1) | (df.hol_Reg == 1) | (df.hol_Loc == 1)  ]
df_hol = df_hol[['date', 'year','month','day','day_of_week','city','state','hol_Nat','hol_Nat_name',
                 'hol_Reg','hol_Reg_name','hol_Loc','hol_loc_name','transferred']]
df_hol = df_hol.drop_duplicates() 
# Check if there's holiday in Aug after 15: 
print(df_hol[(df_hol.month == 8) & (df_hol.day > 15)])

               date  year  month  day  day_of_week    city       state  \
419265   2013-08-24  2013      8   24            5  Ambato  Tungurahua   
1067913  2014-08-24  2014      8   24            6  Ambato  Tungurahua   
1716561  2015-08-24  2015      8   24            0  Ambato  Tungurahua   
2366991  2016-08-24  2016      8   24            2  Ambato  Tungurahua   

         hol_Nat hol_Nat_name  hol_Reg hol_Reg_name  hol_Loc  \
419265       0.0          NaN      0.0          NaN      1.0   
1067913      0.0          NaN      0.0          NaN      1.0   
1716561      0.0          NaN      0.0          NaN      1.0   
2366991      0.0          NaN      0.0          NaN      1.0   

                hol_loc_name  transferred  
419265   Fundacion de Ambato        False  
1067913  Fundacion de Ambato        False  
1716561  Fundacion de Ambato        False  
2366991  Fundacion de Ambato        False  


In [67]:
#Confirm with the orginal holiday csv:
holidays[holidays['month-day'] == '08-24']

,date,type,locale,locale_name,description,transferred,year,month-day
16,2012-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2012,08-24
69,2013-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2013,08-24
132,2014-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2014,08-24
187,2015-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2015,08-24
271,2016-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2016,08-24
327,2017-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2017,08-24


In [68]:
# Check how many stores are affected.
df[df.hol_loc_name == 'Fundacion de Ambato'].store_nbr.unique()

array([23, 50])

## Observation 1:
For kaggle competition purposes: 

We only have to predict for 2017 Aug 16 - Aug 31, only **Aug 24** is a local holiday. 

And it is a **Thursday**. 

Only the city Ambato is affected with stores #23 and #50. 

## Other holiday affects 

### 1. Do all holidays affect sales for all the items in the same way?

Only look into the holidays that are celebrated. Not inlucding the transferred/bridge/workdays 

In [69]:
df_hol_all = df[(df.hol_Nat == 1) | (df.hol_Reg == 1) | (df.hol_Loc == 1)  ]
df_hol_true = df_hol_all[(df_hol_all.hol_type_Holiday == 1) & (df_hol_all.transferred.isin([False, 'False']) ) ]  # Only those holidays + celebrated


In [70]:
# Check holidays:
df_hol_all.transferred.unique()
# The transferred is not right. So for the following code I figured it manually

array([False,  True])

In [71]:
# print(df_hol_true)
utility.summary(df_hol_all)

data shape: (173415, 39)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
Unnamed: 0,int64,0,0.000000,173415,0.0,2999303.0,0,1,2
date,object,0,0.000000,203,NaN,NaN,2013-01-01,2013-01-01,2013-01-01
year,int64,0,0.000000,5,2013.0,2017.0,2013,2013,2013
month,int64,0,0.000000,12,1.0,12.0,1,1,1
week_number,int64,0,0.000000,38,1.0,53.0,1,1,1
day,int64,0,0.000000,29,1.0,31.0,1,1,1
day_of_week,int64,0,0.000000,7,0.0,6.0,1,1,1
store_nbr,int64,0,0.000000,54,1.0,54.0,1,1,1
type,object,0,0.000000,5,NaN,NaN,D,D,D
cluster,int64,0,0.000000,17,1.0,17.0,13,13,13


In [72]:
# Calculate the average values for all the items when not holidays or events 
df_non_hol = df[(df.hol_Nat == 0) | (df.hol_Reg == 0) | (df.hol_Loc == 0) ]
df_sale = pd.DataFrame(columns=['family', 'average_sales_non_hol'])
# Loop through each unique family in the filtered dataframe
# Initialize an empty list to collect results
results = []
for item in df_non_hol['family'].unique():
    # Calculate the mean sales for each family and add it to the dataframe
    avg_sales = np.nanmean(df_non_hol[df_non_hol['family'] == item]['sales'])
    # Append a dictionary with 'family' and 'average_sales' to results list
    results.append({'family': item, 'average_sales': avg_sales})

# Convert the results list to a DataFrame
df_sale = pd.DataFrame(results)
# print(df_sale)

In [73]:
# Add that for holidays 
# National is too complicated. start from Regional
df_sale_regional = df_sale
for hol in df_hol_true.hol_Reg_name.unique():
    if  pd.isna(hol):
        print("Empty holiday name:", hol)
        continue
    # Calculate the average for each item: 
    for item in df_non_hol['family'].unique():
        average_sales = np.nanmean(df_hol_true[(df_hol_true.family == item) | (df_hol_true.hol_Reg_name == hol)]['sales'])
        df_sale_regional.loc[df_sale_regional['family'] == item, hol] = average_sales
#print(df_sale)

Empty holiday name: nan


In [74]:
df_sale_regional 



,family,average_sales,Provincializacion de Cotopaxi,Provincializacion de Imbabura,Provincializacion de Santo Domingo,Provincializacion Santa Elena
0,AUTOMOTIVE,6.101236,34.706042,16.616036,43.791446,16.188166
1,BABY CARE,0.110528,28.698258,10.113243,37.921062,9.689363
2,BEAUTY,3.715723,32.237176,13.929683,41.367478,13.500616
3,BEVERAGES,2385.793151,2053.653410,2193.730960,2014.397166,2192.791119
4,BOOKS,0.070797,28.700482,10.118430,37.923234,9.691757
5,BREAD/BAKERY,463.336254,408.572537,419.710929,408.517403,419.143085
6,CELEBRATION,8.370469,34.291661,16.156738,43.370736,15.730464
7,CLEANING,1072.416744,948.435100,1003.273658,936.112952,1003.427591
8,DAIRY,709.154889,627.747183,656.114839,622.660019,655.423202
9,DELI,265.135067,264.170050,264.139537,267.791623,263.879919


The Comparing doesn't really mean anything especially for some items the average is really low. 

I think for regional average, it's more fair to only consider the regions average for the stores invovled. 

In [77]:
utility.summary(df_nonhol_for_Hol_region)

data shape: (389004, 39)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
Unnamed: 0,int64,0,0.000000,389004,99.0,3000590.0,99,100,101
date,object,0,0.000000,1684,NaN,NaN,2013-01-01,2013-01-01,2013-01-01
year,int64,0,0.000000,5,2013.0,2017.0,2013,2013,2013
month,int64,0,0.000000,12,1.0,12.0,1,1,1
week_number,int64,0,0.000000,53,1.0,53.0,1,1,1
day,int64,0,0.000000,31,1.0,31.0,1,1,1
day_of_week,int64,0,0.000000,7,0.0,6.0,1,1,1
store_nbr,int64,0,0.000000,7,5.0,25.0,12,12,12
type,object,0,0.000000,3,NaN,NaN,C,C,C
cluster,int64,0,0.000000,5,1.0,15.0,15,15,15


In [76]:
# Re do the average for only the stores invovled. 
# First take out regions
region_list = df[df.hol_Reg == 1].store_nbr.unique()
df_nonhol_for_Hol_region = df[df.store_nbr.isin(region_list)]



df_sale_regional = pd.DataFrame(columns=['family', 'average_sales_non_hol'])
# Loop through each unique family in the filtered dataframe
# Initialize an empty list to collect results
results = []
for item in df_nonhol_for_Hol_region['family'].unique():
    # Calculate the mean sales for each family and add it to the dataframe
    avg_sales = np.nanmean(df_nonhol_for_Hol_region[df_nonhol_for_Hol_region['family'] == item]['sales'])
    # Append a dictionary with 'family' and 'average_sales' to results list
    results.append({'family': item, 'average_sales': avg_sales})

# Convert the results list to a DataFrame
df_sale_regional = pd.DataFrame(results)
# print(df_sale)

In [78]:
# Add that for holidays 
# National is too complicated. start from Regional

for hol in df_hol_true.hol_Reg_name.unique():
    if  pd.isna(hol):
        print("Empty holiday name:", hol)
        continue
    # Calculate the average for each item: 
    for item in df_non_hol['family'].unique():
        average_sales = np.nanmean(df_hol_true[(df_hol_true.family == item) | (df_hol_true.hol_Reg_name == hol)]['sales'])
        df_sale_regional.loc[df_sale_regional['family'] == item, hol] = average_sales
#print(df_sale)

Empty holiday name: nan


In [79]:
df_sale_regional

,family,average_sales,Provincializacion de Cotopaxi,Provincializacion de Imbabura,Provincializacion de Santo Domingo,Provincializacion Santa Elena
0,AUTOMOTIVE,4.414489,34.706042,16.616036,43.791446,16.188166
1,BABY CARE,0.224975,28.698258,10.113243,37.921062,9.689363
2,BEAUTY,2.107058,32.237176,13.929683,41.367478,13.500616
3,BEVERAGES,1334.108755,2053.653410,2193.730960,2014.397166,2192.791119
4,BOOKS,0.030115,28.700482,10.118430,37.923234,9.691757
5,BREAD/BAKERY,227.678672,408.572537,419.710929,408.517403,419.143085
6,CELEBRATION,5.749152,34.291661,16.156738,43.370736,15.730464
7,CLEANING,779.393875,948.435100,1003.273658,936.112952,1003.427591
8,DAIRY,341.235579,627.747183,656.114839,622.660019,655.423202
9,DELI,193.849648,264.170050,264.139537,267.791623,263.879919


### Summary on regional holidays: 

There's a huge amount of sales improvement for almosty all the holidays. There inference is some what similar. 

Check the some for the local festivals 

In [80]:
# First take out regions
local_list = df[df.hol_Loc == 1].store_nbr.unique()
df_nonhol_for_Hol_local = df[df.store_nbr.isin(local_list)]



df_sale_local = pd.DataFrame(columns=['family', 'average_sales_non_hol'])
# Loop through each unique family in the filtered dataframe
# Initialize an empty list to collect results
results = []
for item in df_nonhol_for_Hol_local['family'].unique():
    # Calculate the mean sales for each family and add it to the dataframe
    avg_sales = np.nanmean(df_nonhol_for_Hol_local[df_nonhol_for_Hol_local['family'] == item]['sales'])
    # Append a dictionary with 'family' and 'average_sales' to results list
    results.append({'family': item, 'average_sales': avg_sales})

# Convert the results list to a DataFrame
df_sale_local = pd.DataFrame(results)
# print(df_sale)

In [81]:
# Add that for holidays 
# National is too complicated. start from Regional

for hol in df_hol_true.hol_loc_name.unique():
    if  pd.isna(hol):
        print("Empty holiday name:", hol)
        continue
    # Calculate the average for each item: 
    for item in df_non_hol['family'].unique():
        average_sales = np.nanmean(df_hol_true[(df_hol_true.family == item) | (df_hol_true.hol_loc_name == hol)]['sales'])
        df_sale_local.loc[df_sale_local['family'] == item, hol] = average_sales
#print(df_sale)

Empty holiday name: nan


In [82]:
df_sale_local

,family,average_sales,Fundacion de Manta,Fundacion de Cuenca,Cantonizacion de Libertad,Cantonizacion de Riobamba,Cantonizacion de Guaranda,Cantonizacion de Latacunga,Fundacion de Machala,Fundacion de Santo Domingo,...,Fundacion de Ambato,Fundacion de Ibarra,Cantonizacion de Quevedo,Independencia de Guaranda,Independencia de Latacunga,Independencia de Ambato,Fundacion de Quito,Fundacion de Loja,Cantonizacion del Puyo,Fundacion de Guayaquil-1
0,AUTOMOTIVE,6.130082,15.462617,50.974807,21.482143,15.602857,18.056984,25.532125,38.247349,48.048273,...,38.892751,18.057616,17.146760,17.803192,23.333275,29.944381,402.534265,29.826028,8.923861,33.719465
1,BABY CARE,0.104420,9.434077,45.297410,15.072372,9.105650,11.648395,19.365078,32.077645,42.378924,...,32.732158,11.558813,10.651947,11.309975,17.158256,23.617922,399.219656,23.332812,2.338169,27.542169
2,BEAUTY,3.809080,12.977813,48.636808,18.839511,12.920893,15.415140,22.992261,35.696476,45.716223,...,36.347192,15.373658,14.467590,15.123623,20.787337,27.331930,401.099195,27.138079,6.204378,31.168592
3,BEVERAGES,2421.625448,2037.977443,1953.979005,2169.766225,2193.680271,2166.836732,2094.988844,2105.084859,1950.702577,...,2106.984246,2195.261926,2194.553783,2193.580127,2093.171164,2150.586793,1482.162843,2205.766172,2215.743586,2101.340574
4,BOOKS,0.073797,9.436301,45.300559,15.074736,9.108044,11.650759,19.367736,32.080303,42.381723,...,32.734436,11.561208,10.654342,11.312370,17.160534,23.620256,399.217948,23.334807,2.340595,27.544446
5,BREAD/BAKERY,467.863787,389.683834,402.217528,419.000190,418.616745,416.017112,408.717699,420.729420,400.155697,...,421.350844,421.196404,420.174215,420.898484,406.538591,422.460437,606.044093,432.241433,417.433886,416.089879
6,CELEBRATION,8.534151,15.043788,50.577326,21.038881,15.150343,17.590081,25.095526,37.800121,47.630498,...,38.442105,17.600314,16.669106,17.351077,22.886046,29.484459,402.191677,29.364336,8.452155,33.254393
7,CLEANING,1086.234782,931.954833,913.002098,995.473081,1001.812753,992.254384,963.323317,974.749627,909.595159,...,975.351748,1003.088741,1003.829122,1004.472785,961.113077,991.249439,900.951608,1015.825230,1009.403246,971.421060
8,DAIRY,720.703460,609.356983,609.054932,652.442348,655.045794,649.484486,633.392793,645.332391,607.116852,...,645.690017,657.531678,656.667111,657.256504,631.295310,652.537377,720.872155,668.059468,657.018040,640.861075
9,DELI,269.395841,245.511835,267.716196,265.933058,263.160675,262.393822,260.816063,273.391537,264.275725,...,273.921015,265.522857,264.810724,265.307938,258.644548,271.144974,527.159129,277.428414,259.950899,268.906912


In [83]:
# THe local holidays are not right. 
df_hol_all[df_hol_all.hol_Loc == 1].transferred.unique()

array([False])

In [84]:
Local[Local.type == 'Holiday'].description.unique()

array(['Fundacion de Manta', 'Fundacion de Cuenca',
       'Cantonizacion de Libertad', 'Cantonizacion de Riobamba',
       'Cantonizacion del Puyo', 'Cantonizacion de Guaranda',
       'Cantonizacion de Latacunga', 'Fundacion de Machala',
       'Fundacion de Santo Domingo', 'Cantonizacion de El Carmen',
       'Cantonizacion de Cayambe', 'Fundacion de Esmeraldas',
       'Fundacion de Riobamba', 'Fundacion de Ambato',
       'Fundacion de Ibarra', 'Cantonizacion de Quevedo',
       'Independencia de Guaranda', 'Independencia de Latacunga',
       'Independencia de Ambato', 'Fundacion de Quito',
       'Fundacion de Loja', 'Cantonizacion de Salinas',
       'Fundacion de Guayaquil', 'Fundacion de Guayaquil-1'], dtype=object)

For Some reason I'm not having all the holidays I'm supposed to have. I suspect it's the transferred label that's not right. 

# Other checks on events. 


In [88]:
holidays[holidays.type == 'Event']

,date,type,locale,locale_name,description,transferred,year,month-day
55,2013-05-12,Event,National,Ecuador,Dia de la Madre,False,2013,05-12
103,2014-05-11,Event,National,Ecuador,Dia de la Madre,False,2014,05-11
106,2014-06-12,Event,National,Ecuador,Inauguracion Mundial de futbol Brasil,False,2014,06-12
107,2014-06-15,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Suiza,False,2014,06-15
108,2014-06-20,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Honduras,False,2014,06-20
113,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False,2014,06-25
114,2014-06-28,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,2014,06-28
115,2014-06-29,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,2014,06-29
116,2014-06-30,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,2014,06-30
117,2014-07-01,Event,National,Ecuador,Mundial de futbol Brasil: Octavos de Final,False,2014,07-01


In [86]:
national 

,date,type,locale,locale_name,description,transferred,year,month-day
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,2012,08-10
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,2012,10-09
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False,2012,11-02
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False,2012,11-03
37,2012-12-25,Holiday,National,Ecuador,Navidad,False,2012,12-25
41,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False,2013,01-01
44,2013-02-11,Holiday,National,Ecuador,Carnaval,False,2013,02-11
45,2013-02-12,Holiday,National,Ecuador,Carnaval,False,2013,02-12
51,2013-04-29,Holiday,National,Ecuador,Viernes Santo,False,2013,04-29
52,2013-05-01,Holiday,National,Ecuador,Dia del Trabajo,False,2013,05-01
